In [1]:
import tweepy           
import pandas as pd    
import numpy as np
from datetime import datetime
import preprocessor as p
import datetime
import xlsxwriter
import sys
from tweepy.streaming import StreamListener

In [2]:
df1 = pd.read_csv("All_Trump_Tweets.csv")
df1.head()

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
0,Twitter for iPhone,"RT @WhiteHouse: ""America is engaged in a histo...",4/3/2020 23:51,12244.0,0.0,True,1.246220e+18
1,Twitter for iPhone,RT @WhiteHouse: Our hearts go out to the commu...,4/3/2020 23:50,7254.0,0.0,True,1.246220e+18
2,Twitter for iPhone,RT @WhiteHouse: LIVE: Press Briefing with Coro...,4/3/2020 21:24,6807.0,0.0,True,1.246190e+18
3,Twitter for iPhone,Great job being done by @BankofAmerica and man...,4/3/2020 18:49,16337.0,96640.0,False,1.246150e+18
4,Twitter for iPhone,The Paycheck Protection Program is now availab...,4/3/2020 17:51,13350.0,45408.0,False,1.246130e+18


In [9]:
adc = df1.drop(columns=['id_str', 'is_retweet'])

In [10]:
adc = adc.rename(columns={"created_at": "Date", "text": "Tweet", "retweet_count": "Retweets", "favorite_count": "Favorites"})
adc.head()

,source,Tweet,Date,Retweets,Favorites
0,Twitter for iPhone,"RT @WhiteHouse: ""America is engaged in a histo...",4/3/2020 23:51,12244.0,0.0
1,Twitter for iPhone,RT @WhiteHouse: Our hearts go out to the commu...,4/3/2020 23:50,7254.0,0.0
2,Twitter for iPhone,RT @WhiteHouse: LIVE: Press Briefing with Coro...,4/3/2020 21:24,6807.0,0.0
3,Twitter for iPhone,Great job being done by @BankofAmerica and man...,4/3/2020 18:49,16337.0,96640.0
4,Twitter for iPhone,The Paycheck Protection Program is now availab...,4/3/2020 17:51,13350.0,45408.0


In [13]:
adc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 5 columns):
source       16715 non-null object
Tweet        16715 non-null object
Date         16713 non-null object
Retweets     16713 non-null float64
Favorites    16713 non-null float64
dtypes: float64(2), object(3)
memory usage: 653.1+ KB


In [14]:
adc['Date'] = pd.to_datetime(adc['Date'])

In [15]:
adc['New Date'] = adc.Date+pd.offsets.DateOffset(hours=-5)
adc.head()

,source,Tweet,Date,Retweets,Favorites,New Date
0,Twitter for iPhone,"RT @WhiteHouse: ""America is engaged in a histo...",2020-04-03 23:51:00,12244.0,0.0,2020-04-03 18:51:00
1,Twitter for iPhone,RT @WhiteHouse: Our hearts go out to the commu...,2020-04-03 23:50:00,7254.0,0.0,2020-04-03 18:50:00
2,Twitter for iPhone,RT @WhiteHouse: LIVE: Press Briefing with Coro...,2020-04-03 21:24:00,6807.0,0.0,2020-04-03 16:24:00
3,Twitter for iPhone,Great job being done by @BankofAmerica and man...,2020-04-03 18:49:00,16337.0,96640.0,2020-04-03 13:49:00
4,Twitter for iPhone,The Paycheck Protection Program is now availab...,2020-04-03 17:51:00,13350.0,45408.0,2020-04-03 12:51:00


In [16]:
adc['New Date'] = pd.to_datetime(adc['New Date'])

In [17]:
adc['New Date'] = adc['New Date'].dt.strftime('%Y%m%d %H%M')

In [18]:
adc.head()

,source,Tweet,Date,Retweets,Favorites,New Date
0,Twitter for iPhone,"RT @WhiteHouse: ""America is engaged in a histo...",2020-04-03 23:51:00,12244.0,0.0,20200403 1851
1,Twitter for iPhone,RT @WhiteHouse: Our hearts go out to the commu...,2020-04-03 23:50:00,7254.0,0.0,20200403 1850
2,Twitter for iPhone,RT @WhiteHouse: LIVE: Press Briefing with Coro...,2020-04-03 21:24:00,6807.0,0.0,20200403 1624
3,Twitter for iPhone,Great job being done by @BankofAmerica and man...,2020-04-03 18:49:00,16337.0,96640.0,20200403 1349
4,Twitter for iPhone,The Paycheck Protection Program is now availab...,2020-04-03 17:51:00,13350.0,45408.0,20200403 1251


In [19]:
from textblob import TextBlob
import re

def clean_tweet(tweet):
    '''
    Utility function to clean the text in a tweet by removing 
    links and special characters using regex.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def analize_sentiment(tweet):
    '''
    Utility function to classify the polarity of a tweet
    using textblob.
    '''
    analysis = TextBlob(clean_tweet(tweet))
    if analysis.sentiment.polarity > 0:
        return 1
    elif analysis.sentiment.polarity == 0:
        return 0
    else:
        return -1

In [22]:
adc['SA'] = np.array([ analize_sentiment(tweet) for tweet in adc['Tweet'] ])

In [23]:
adc.head()

,source,Tweet,Date,Retweets,Favorites,New Date,SA
0,Twitter for iPhone,"RT @WhiteHouse: ""America is engaged in a histo...",2020-04-03 23:51:00,12244.0,0.0,20200403 1851,0
1,Twitter for iPhone,RT @WhiteHouse: Our hearts go out to the commu...,2020-04-03 23:50:00,7254.0,0.0,20200403 1850,1
2,Twitter for iPhone,RT @WhiteHouse: LIVE: Press Briefing with Coro...,2020-04-03 21:24:00,6807.0,0.0,20200403 1624,1
3,Twitter for iPhone,Great job being done by @BankofAmerica and man...,2020-04-03 18:49:00,16337.0,96640.0,20200403 1349,1
4,Twitter for iPhone,The Paycheck Protection Program is now availab...,2020-04-03 17:51:00,13350.0,45408.0,20200403 1251,1


In [24]:
adf.to_csv('FinalTrumpTweet3.csv')